<a href="https://colab.research.google.com/github/Shaymaxo/Capstone-2-Springboard/blob/main/4_modeling_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal: Develop a final model that predicts fraudulent transactions using two to three different algorithms, apply hyperparameter tuning, and define metrics for model selection.

## Modeling Setup and Metrics Definition


**Modeling Goal:** Build and compare multiple classification models.

**Algorithms:** Logistic Regression, Random Forest, XGBoost

**Hyperparameter Tuning:** GridSearchCV with cross-validation

**Evaluation Metrics:** Accuracy, Precision, Recall, F1-score, ROC AUC

**Final Selection Criteria:** Balance between predictive performance, computational efficiency, and scalability.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Base directory where your project folders live
drive_base = '/content/drive/MyDrive'
# Path to the folder containing your EDA notebook
pre_processing_folder = os.path.join(drive_base, 'data', 'raw', 'Capstone 2 - Data Wrangling')
pre_processing_notebook_path = os.path.join(pre_processing_folder, '3. pre-processing capstone.ipynb')
print('Pre-processing notebook path:', pre_processing_notebook_path)

Pre-processing notebook path: /content/drive/MyDrive/data/raw/Capstone 2 - Data Wrangling/3. pre-processing capstone.ipynb


In [ ]:
# %%
# 2. Imports and Configuration
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, precision_recall_curve)
from sklearn.decomposition import PCA
from scipy.sparse import issparse

In [ ]:
# %% [markdown]
## 3. Load and Sample Data

# %%
data_path = "/content/drive/MyDrive/Capstone1/Capstone 2 - Data Wrangling/ieee-fraud-detection_project/data/raw/processed_fraud_data.csv"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data not found at {data_path}")

df = pd.read_csv(data_path)
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
_, df_sampled = train_test_split(df, train_size=40000, stratify=df['isFraud'], random_state=42)
X_full = df_sampled.drop(columns=['isFraud'])
y_full = df_sampled['isFraud']
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, stratify=y_full, random_state=42
)
print("Training set:", X_train.shape, "Test set:", X_test.shape)

Training set: (48000, 433) Test set: (12000, 433)


The dataset was successfully loaded and stratified to maintain the original class distribution of fraudulent vs. non-fraudulent transactions. A sample of 40,000 transactions was used to optimize performance, then split into training and test sets in an 80/20 ratio. This resulted in 48,000 training samples and 12,000 test samples, each with 433 features, ensuring a balanced and efficient foundation for modeling.

In [ ]:
# %% [markdown]
## 4. Preprocessing and Dimensionality Reduction

# Identify column types
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                                 ('scaler', StandardScaler())])
cat_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                            ('encoder', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

def transform_dense(prep, X, fit=True):
    Xt = prep.fit_transform(X) if fit else prep.transform(X)
    if issparse(Xt):
        Xt = Xt.toarray()
    return Xt.astype(np.float32)

X_train_prep = transform_dense(preprocessor, X_train, fit=True)
X_test_prep = transform_dense(preprocessor, X_test, fit=False)

# Apply PCA to speed up baseline
pca = PCA(n_components=50, random_state=42)
X_train_reduced = pca.fit_transform(X_train_prep)
X_test_reduced = pca.transform(X_test_prep)
print(f"Reduced features: {X_train_reduced.shape[1]}")

Reduced features: 50


The data was successfully preprocessed using imputation, scaling, and one-hot encoding to handle missing values and categorical features. To improve computational efficiency, PCA was applied, reducing the feature space from 433 to 50 components while preserving key variance, making the dataset more manageable for baseline model training.

In [ ]:
# %% [markdown]
## 5. Baseline Models (No Tuning) on Reduced Data

# Switch to liblinear for faster Logistic Regression
models = {
    'LogisticRegression': LogisticRegression(solver='liblinear', max_iter=200, class_weight='balanced', random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=50, max_depth=10, class_weight='balanced', random_state=42),
    'XGBoost': XGBClassifier(n_estimators=50, max_depth=6, learning_rate=0.1, subsample=0.7,
                              colsample_bytree=0.7, scale_pos_weight=1, use_label_encoder=False,
                              eval_metric='logloss', random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train_reduced, y_train)
    y_pred = model.predict(X_test_reduced)
    y_proba = model.predict_proba(X_test_reduced)[:,1]
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_proba)
    }

baseline_df = pd.DataFrame(results).T
print(baseline_df)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:36:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


                    Accuracy  Precision    Recall        F1   ROC AUC
LogisticRegression  0.777417   0.078760  0.719870  0.141985  0.817217
RandomForest        0.945333   0.228616  0.478827  0.309474  0.852522
XGBoost             0.978417   0.875000  0.182410  0.301887  0.847730


Three baseline models—Logistic Regression, Random Forest, and XGBoost—were evaluated on the reduced dataset. Logistic Regression achieved **high recall (0.72)** but suffered from **low precision and F1 score**, indicating many false positives. Random Forest and XGBoost **outperformed in overall accuracy (94.5% and 97.8%)** and AUC, with XGBoost showing **very high precision (0.88)** but **low recall**, suggesting it's more conservative in flagging fraud. This highlights the classic trade-off between detecting fraud and minimizing false alarms.

In [ ]:
# %% [markdown]
## 6. Hyperparameter Tuning (RandomizedSearchCV)

# Tune only RandomForest with a slightly broader grid
from scipy.stats import randint

rf_param_dist = {
    'n_estimators': randint(50, 100),
    'max_depth': [8, 10, 12],
    'class_weight': ['balanced', 'balanced_subsample']
}

ts = RandomizedSearchCV(
    models['RandomForest'],
    param_distributions=rf_param_dist,
    n_iter=10,
    scoring='roc_auc',
    cv=2,
    n_jobs=-1,
    random_state=42
)
ts.fit(X_train_reduced, y_train)

# Evaluate tuned RandomForest
tuned_rf = ts.best_estimator_
y_pred_rf = tuned_rf.predict(X_test_reduced)
y_proba_rf = tuned_rf.predict_proba(X_test_reduced)[:,1]

random_results = {
    'RandomForest': {
        'BestParams': ts.best_params_,
        'Accuracy': accuracy_score(y_test, y_pred_rf),
        'Precision': precision_score(y_test, y_pred_rf),
        'Recall': recall_score(y_test, y_pred_rf),
        'F1': f1_score(y_test, y_pred_rf),
        'ROC AUC': roc_auc_score(y_test, y_proba_rf)
    }
}
random_df = pd.DataFrame(random_results).T
print(random_df)

# Adjust threshold to improve recall
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_rf)
target_recall = 0.6
best_idx = np.argmax(recalls >= target_recall)
if best_idx < len(thresholds):
    custom_thresh = thresholds[best_idx]
    print(f"Selected threshold: {custom_thresh:.2f} for recall: {recalls[best_idx]:.3f}")

    y_pred_thresh = (y_proba_rf >= custom_thresh).astype(int)
    print("Adjusted threshold metrics:")
    print("Accuracy:", accuracy_score(y_test, y_pred_thresh))
    print("Precision:", precision_score(y_test, y_pred_thresh))
    print("Recall:", recall_score(y_test, y_pred_thresh))
    print("F1:", f1_score(y_test, y_pred_thresh))
    print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))
else:
    print("No threshold found for desired recall.")

                                                     BestParams  Accuracy  \
RandomForest  {'class_weight': 'balanced', 'max_depth': 8, '...  0.901583   

             Precision    Recall       F1  ROC AUC  
RandomForest  0.144715  0.579805  0.23162  0.84229  
Selected threshold: 0.06 for recall: 1.000
Adjusted threshold metrics:
Accuracy: 0.025583333333333333
Precision: 0.025583333333333333
Recall: 1.0
F1: 0.04989030632973105
ROC AUC: 0.8422898969872841


Hyperparameter tuning of the Random Forest modestly improved its detection ability—**raising recall to 0.58 while holding ROC AUC at 0.84**—at the expense of some precision (0.14). Pushing the decision threshold to achieve perfect recall (1.0) confirmed the extreme trade-off: precision and accuracy both fell to ~2.6%, illustrating that maximizing fraud capture can overwhelm false positives. This underscores the need to balance threshold selection to meet real-world business requirements.

In [ ]:
# -------------------------------
# Threshold fine-tuning on tuned RandomForest
probas = y_proba_rf
thresholds = np.linspace(0, 1, 101)
best_f1 = 0.0
best_thresh = 0.5

for t in thresholds:
    y_pred_t = (probas >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold by F1: {best_thresh:.2f} with F1: {best_f1:.4f}")

# Evaluate at best threshold
y_pred_thresh = (probas >= best_thresh).astype(int)
print("Metrics at best threshold:")
print("Accuracy:", accuracy_score(y_test, y_pred_thresh))
print("Precision:", precision_score(y_test, y_pred_thresh))
print("Recall:", recall_score(y_test, y_pred_thresh))
print("F1:", f1_score(y_test, y_pred_thresh))
print("ROC AUC (proba):", roc_auc_score(y_test, probas))
# -------------------------------

# %% [markdown]
## 7. Ensemble Model: VotingClassifier

from sklearn.ensemble import VotingClassifier

# Create a soft-voting ensemble of the tuned RandomForest and your XGBoost model
ensemble = VotingClassifier(
    estimators=[
        ('rf', tuned_rf),
        ('xgb', models['XGBoost'])  # use your baseline XGBoost or a tuned version
    ],
    voting='soft',  # uses predicted probabilities
    n_jobs=-1
)

ensemble.fit(X_train_reduced, y_train)
y_pred_ens = ensemble.predict(X_test_reduced)
y_proba_ens = ensemble.predict_proba(X_test_reduced)[:, 1]

ensemble_results = {
    'Ensemble': {
        'Accuracy': accuracy_score(y_test, y_pred_ens),
        'Precision': precision_score(y_test, y_pred_ens),
        'Recall': recall_score(y_test, y_pred_ens),
        'F1': f1_score(y_test, y_pred_ens),
        'ROC AUC': roc_auc_score(y_test, y_proba_ens)
    }
}

ensemble_df = pd.DataFrame(ensemble_results).T
print(ensemble_df)

Best threshold by F1: 0.69 with F1: 0.3872
Metrics at best threshold:
Accuracy: 0.9744166666666667
Precision: 0.5
Recall: 0.31596091205211724
F1: 0.3872255489021956
ROC AUC (proba): 0.8422898969872841
          Accuracy  Precision   Recall        F1   ROC AUC
Ensemble   0.97975   0.796296  0.28013  0.414458  0.846561


Optimizing the Random Forest decision threshold by F1 score identified 0.69 as the sweet spot—boosting F1 to 0.387 with 50% precision and 32% recall while maintaining ROC AUC at 0.842. Building a soft-voting ensemble of this tuned RF and the XGBoost model further improved overall separation (ROC AUC 0.847) and F1 (0.414), with precision climbing to 0.80 though recall dipped to 0.28. In practice, the threshold-tuned RF balances fraud detection and false alarms effectively, while the ensemble offers higher confidence in flagged cases at the expense of missing some frauds.


In [ ]:
# %% [markdown]
## 7.1 Ensemble Model (Voting)

#To balance precision and recall, we build a soft-voting ensemble of the tuned RandomForest and a tuned XGBoost (using the same reduced data and fast-mode parameters).

from sklearn.ensemble import VotingClassifier

# Re-initialize XGBoost with fast-mode tuned parameters
xgb_fast = XGBClassifier(
    n_estimators=30, max_depth=6, learning_rate=0.1,
    subsample=0.7, colsample_bytree=0.7, scale_pos_weight=1,
    use_label_encoder=False, eval_metric='logloss', random_state=42
)

# Fit XGBoost on reduced data
xgb_fast.fit(X_train_reduced, y_train)

# Assemble voting ensemble
ensemble = VotingClassifier(
    estimators=[('rf', tuned_rf), ('xgb', xgb_fast)],
    voting='soft', weights=[1,1], n_jobs=-1
)

ensemble.fit(X_train_reduced, y_train)

# Evaluate ensemble
y_pred_ens = ensemble.predict(X_test_reduced)
y_proba_ens = ensemble.predict_proba(X_test_reduced)[:,1]

print("Ensemble Classification Report:")
print(classification_report(y_test, y_pred_ens))
print("Ensemble ROC AUC:", roc_auc_score(y_test, y_proba_ens))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:28:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Ensemble Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     11693
           1       0.82      0.24      0.38       307

    accuracy                           0.98     12000
   macro avg       0.90      0.62      0.68     12000
weighted avg       0.98      0.98      0.97     12000

Ensemble ROC AUC: 0.8452512444456453


The soft-voting ensemble of the tuned Random Forest and a fast-mode XGBoost model achieved a strong overall ROC AUC of 0.845, with precision at 82%—meaning most flagged transactions are true frauds—while recall remained modest at 24%, indicating it still misses a majority of fraud cases. Its F1-score of 0.38 reflects this trade-off, balancing high confidence in detected fraud against lower coverage. This ensemble is well suited for scenarios where prioritizing precision (i.e., minimizing false alarms) is critical, though further threshold tuning or additional techniques would be needed to improve recall if catching a higher fraction of frauds is the primary goal.

In [ ]:
# %% [markdown]
## 8. SMOTE + RandomForest Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Build an imbalanced-aware pipeline
smote_rf_pipe = ImbPipeline([
    ('smote', SMOTE(sampling_strategy=0.5, random_state=42)),
    # 0.5 means after SMOTE, fraud samples = 50% of non‐fraud
    ('rf', RandomForestClassifier(
        n_estimators=80, max_depth=10,
        class_weight='balanced_subsample',
        random_state=42, n_jobs=-1
    ))
])

# Fit on the original reduced training data
smote_rf_pipe.fit(X_train_reduced, y_train)

# Evaluate on the test set
y_pred_smote = smote_rf_pipe.predict(X_test_reduced)
y_proba_smote = smote_rf_pipe.predict_proba(X_test_reduced)[:,1]

print("SMOTE + RF Classification Report:")
print(classification_report(y_test, y_pred_smote))
print("ROC AUC:", roc_auc_score(y_test, y_proba_smote))


SMOTE + RF Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.89      0.94     11693
           1       0.13      0.62      0.21       307

    accuracy                           0.88     12000
   macro avg       0.56      0.75      0.57     12000
weighted avg       0.97      0.88      0.92     12000

ROC AUC: 0.8494792535749696


By applying SMOTE to oversample the minority class to 50% of the majority and then fitting a Random Forest, recall was substantially increased to 62%, ensuring the model captures more fraud instances. However, precision dropped to 13%, reflecting many false positives, and overall accuracy fell to 88%. The ROC AUC improved slightly to 0.849, indicating better class separation. This approach is valuable when maximizing fraud detection (recall) is paramount, but may require further tuning or downstream filtering to manage the higher false‐positive rate.

In [ ]:
# %% [markdown]
## 9. Stacking Ensemble

from sklearn.ensemble import StackingClassifier

# Base learners
estimators = [
    ('lr', models['LogisticRegression']),       # your liblinear LR
    ('rf', tuned_rf),                            # your tuned RF
    ('xgb', models['XGBoost'])                  # your fast‐mode XGBoost
]

stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(solver='liblinear', max_iter=200),
    cv=3,
    n_jobs=-1,
    passthrough=False  # if True, passes original features too
)

# Fit stacking ensemble
stack.fit(X_train_reduced, y_train)

# Evaluate on test set
y_pred_stack = stack.predict(X_test_reduced)
y_proba_stack = stack.predict_proba(X_test_reduced)[:,1]

print("Stacking Ensemble Classification Report:")
print(classification_report(y_test, y_pred_stack))
print("Stacking ROC AUC:", roc_auc_score(y_test, y_proba_stack))


Stacking Ensemble Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     11693
           1       0.84      0.23      0.37       307

    accuracy                           0.98     12000
   macro avg       0.91      0.62      0.68     12000
weighted avg       0.98      0.98      0.97     12000

Stacking ROC AUC: 0.8458597824751634


The stacking ensemble of Logistic Regression, tuned Random Forest, and XGBoost achieved a ROC AUC of 0.846, indicating strong overall discrimination between fraud and non-fraud. It delivered high precision (84%), meaning most flagged transactions are true positives, but modest recall (23%), capturing less than a quarter of actual fraud cases. Its F1 score (0.37) reflects this precision–recall trade-off. This ensemble is ideal when false positives are particularly costly and you need high confidence in each flagged transaction, though additional threshold tuning or complementary methods would be needed to improve its coverage of fraud cases.

In [ ]:
# Create a dictionary for the base and tuned metrics
model_metrics = {
    'LogisticRegression': {
        'Base': {'Accuracy': 0.777417, 'Precision': 0.078760, 'Recall': 0.719870, 'F1': 0.141985, 'ROC AUC': 0.817217},
        'Tuned': {'Accuracy': None, 'Precision': None, 'Recall': None, 'F1': None, 'ROC AUC': None, 'BestParams': None}
    },
    'RandomForest': {
        'Base': {'Accuracy': 0.945333, 'Precision': 0.228616, 'Recall': 0.478827, 'F1': 0.309474, 'ROC AUC': 0.852522},
        'Tuned': {'Accuracy': 0.901583, 'Precision': 0.144715, 'Recall': 0.579805, 'F1': 0.23162, 'ROC AUC': 0.84229, 'BestParams': {'class_weight': 'balanced', 'max_depth': 8, 'n_estimators': 100}}
    },
    'XGBoost': {
        'Base': {'Accuracy': 0.978417, 'Precision': 0.875000, 'Recall': 0.182410, 'F1': 0.301887, 'ROC AUC': 0.847730},
        'Tuned': {'Accuracy': None, 'Precision': None, 'Recall': None, 'F1': None, 'ROC AUC': None, 'BestParams': None}
    },
    'SMOTE+RF': {
        'Base': {'Accuracy': None, 'Precision': None, 'Recall': None, 'F1': None, 'ROC AUC': None},
        'Tuned': {'Accuracy': 0.88, 'Precision': 0.13, 'Recall': 0.62, 'F1': 0.21, 'ROC AUC': 0.8495}
    },
    'Stacking': {
        'Base': {'Accuracy': None, 'Precision': None, 'Recall': None, 'F1': None, 'ROC AUC': None},
        'Tuned': {'Accuracy': 0.98, 'Precision': 0.84, 'Recall': 0.23, 'F1': 0.37, 'ROC AUC': 0.8459}
    }
}

# Convert to DataFrame for comparison
df_compare = pd.DataFrame.from_dict({model: {metric: values['Base'][metric] for metric in values['Base']}
                                     for model, values in model_metrics.items()},
                                    orient='index')

df_tuned = pd.DataFrame.from_dict({model: {metric: values['Tuned'][metric] for metric in values['Tuned']}
                                  for model, values in model_metrics.items()},
                                  orient='index')

# Merge base and tuned DataFrames
df_compare_final = df_compare.join(df_tuned, lsuffix='_Base', rsuffix='_Tuned')

# Display the final DataFrame
print(df_compare_final)


                    Accuracy_Base  Precision_Base  Recall_Base   F1_Base  \
LogisticRegression       0.777417        0.078760     0.719870  0.141985   
RandomForest             0.945333        0.228616     0.478827  0.309474   
XGBoost                  0.978417        0.875000     0.182410  0.301887   
SMOTE+RF                      NaN             NaN          NaN       NaN   
Stacking                      NaN             NaN          NaN       NaN   

                    ROC AUC_Base  Accuracy_Tuned  Precision_Tuned  \
LogisticRegression      0.817217             NaN              NaN   
RandomForest            0.852522        0.901583         0.144715   
XGBoost                 0.847730             NaN              NaN   
SMOTE+RF                     NaN        0.880000         0.130000   
Stacking                     NaN        0.980000         0.840000   

                    Recall_Tuned  F1_Tuned  ROC AUC_Tuned  \
LogisticRegression           NaN       NaN            NaN   
Random

The consolidated results table shows that among all approaches, the threshold‐tuned Random Forest delivers the best balance of metrics for fraud detection: it achieves a solid ROC AUC of 0.842, an F1‐score of 0.232, and improved recall (0.580) over the untuned version, while maintaining reasonable precision. The SMOTE+RF pipeline maximizes recall (0.62) but at the expense of precision (0.13) and overall accuracy, making it useful only when catching nearly all fraud cases is critical. The stacking ensemble delivers the highest precision (0.84) and slightly higher ROC AUC (0.846) but lower recall (0.23), which is suitable when false positives must be minimized. Given the need to both detect a meaningful proportion of frauds and limit false alarms, the threshold‐tuned Random Forest emerges as the most effective final model.

**Final Model Comparison**

After systematically evaluating a variety of models - including base and tuned versions of Logistic Regression, Random Forest, and XGBoost, as well as ensemble methods (Voting, Stacking) and imbalance-handling strategies (SMOTE)—the results reveal key trade-offs between precision, recall, and overall robustness.

*   **Tuned Random Forest** stands out as the **most balanced model**. It achieves a **recall of 0.58**, meaning it captures more than half of the fraudulent transactions, while still preserving reasonable **precision (0.14)** and **ROC AUC (0.84)**. This makes it effective for real-world deployment, where missing too many fraud cases can be costly.
*   **SMOTE+RF** improves recall slightly (0.62) but significantly sacrifices precision (0.13), making it better suited for high-recall use cases like internal alerts - where false positives are acceptable as long as fraud is caught.

*   **Stacking Ensemble** has the highest precision (0.84), but a recall of just 0.23. This model may be better suited when **false positives are costly**, such as in customer-facing scenarios.
*   **Threshold tuning** was a critical step in improving fraud capture rates, highlighting the importance of going beyond default probability thresholds for classification in imbalanced datasets.













# Top 3 Recommended Next Steps


1.   **Deploy the Threshold-Tuned Random Forest Model**: It offers the best balance of fraud detection (recall) and minimizing false positives (precision), making it the most practical choice for production.
2.   **Build and Package a Full Inference Pipeline**: Integrate all pre-processing (e.g., scaling, encoding, feature selection) with the final model to ensure consistent, end-to-end predictions in deployment.

1.   **Validate with End Users**: I recommend collaborating with fraud analysts or domain experts to validate prediction quality, interpretability, and thresholds in real-world use cases.






With these steps, the modeling phase will effectively transition into robust, interpretable, and maintainable fraud-detection operations.